## **Exercise 17.02**
###  Creating New Features Using Deep Feature Synthesis

### Importing modules

In [2]:
import pandas as pd

### Loading data using pandas

In [ ]:
import pandas as pd
auth = pd.read_csv('https://raw.githubusercontent.com/fenago/datasets/main/BankNote_Authentication.csv')
auth.head()

### Removing the target variable

In [29]:

Y = auth.pop('class')

In [30]:
auth['notesID'] = auth.index.values
auth['notesID'] = 'notes' + auth['notesID'].astype(str)

### Creating skewnessId

In [31]:
auth['skewnessId'] = 0
auth.loc[auth.skewness == '1','skewnessId']= 1

### Creating curtosisId

In [32]:
auth['curtosisId'] = 0
auth.loc[auth.curtosis == '1','curtosisId']= 1

### Creating Entropy ID

In [33]:
auth['entropyID'] = 0
auth.loc[auth.entropy == '1','entropyID']= 1

### Displaying the new data frame after adding the ids

In [ ]:
auth.head()

### Installing feature tools

In [ ]:
!pip3  install featuretools

### Importing necessary libraries

In [36]:
import featuretools as ft
import numpy as np

In [37]:
notesEntities = ft.EntitySet(id = 'Notes')

### Mapping a dataframe to the entityset to form the parent entity

In [ ]:
notesEntities.add_dataframe(dataframe_name = 'Variance', dataframe = auth, index = 'notesID')

### Mapping Skewness and setting the relationship

In [ ]:
notesEntities.normalize_dataframe(base_dataframe_name='Variance', new_dataframe_name='Skewness', index = 'skewnessId', additional_columns = ['curtosis'])

### Mapping Curtosis and Entropy entities

In [ ]:
notesEntities.normalize_dataframe(base_dataframe_name='Variance', new_dataframe_name='Curtosis', index = 'curtosisId', additional_columns = ['curtosis'])
notesEntities.normalize_dataframe(base_dataframe_name='Variance', new_dataframe_name='Entropy', index = 'entropyID', additional_columns = ['entropy'])

### Creating feature sets using Deep Feature Synthesis

In [ ]:
feature_set, feature_names = ft.dfs(entityset=notesEntities, target_dataframe_name = 'Variance', \
            max_depth = 2, verbose = 1, n_jobs = 1)


### Reindexing the feature_set

In [42]:
feature_set = feature_set.reindex(index=auth['notesID'])
feature_set = feature_set.reset_index()

### Verifying the shape of the features and original bank data

In [ ]:

print(feature_set.shape)
print(auth.shape)

### Printing head of the feature set

In [ ]:

feature_set.head()

### Printing the list of all features

In [ ]:

feature_names

### Creating aggregation and transformation primitives

In [48]:
aggPrimitives=[
        'std', 'min', 'max', 'mean', 
        'last', 'count' ]
tranPrimitives=[
        'percentile', 
         'subtract_numeric', 'divide_numeric']

### Defining the new set of features

In [ ]:

feature_set, feature_names = ft.dfs(entityset=notesEntities, target_dataframe_name = 'Variance',\
        agg_primitives=aggPrimitives,trans_primitives=tranPrimitives,max_depth = 2, \
                  verbose = 1, n_jobs = 1)